# 00. Инженерия данных и первичная подготовка

## 1. Инициализация окружения и конфигурация

Инициализация путей на основе настроей `config.py`. Если активирован `Сolab`

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import re

In [ ]:
# === Конфигурация ===
class Config:
  """
  Класс для хранения всех настроек проекта
  Если разработка происходит в Colab, изменить IS_COLAB = True
  """
  IS_COLAB = True
  DATA_DIR_NAME = "data"
  FILE_PATTERN = "w{}_25.xlsx"
  WEEKS_RANGE = range(1, 21)

  # Путь к датасет-файлам проекта
  DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Timesheet Analysis/"

In [ ]:
if Config.IS_COLAB:
  try:
    from google.colab import drive
    drive.mount("/content/drive")
    BASE_DIR = Path(Config.DRIVE_PATH) / Config.DATA_DIR_NAME
  except ImportError:
    print("Ошибка подключения к Google Drive")
else:
  BASE_DIR = Path.cwd() / Config.DATA_DIR_NAME

## 2. Загрузка и консолидация данных

Объединяем еженедельные файлы в единый `df`. Имена файлов и количество недель генерируется на основе конфигурационного класса

In [ ]:
def load_timesheet_data(base_dir, weeks_range, pattern):
  """
  Загружает и консолидирует xlsx-файлы за указанные недели
  """
  all_chunks = []
  missing_files =[]

  for week in weeks_range:
    file_path = base_dir / pattern.format(week)

    if not file_path.exists():
      missing_files.append(file_path.name)
      continue

    try:
      df_temp = pd.read_excel(file_path)
      df_temp['source_file'] = file_path.name
      all_chunks.append(df_temp)
    except Exception as e:
      print(f"Ошибка загрузки файла {file_path.name}: {e}")

  if not all_chunks:
    print("Данные не найдены. Необходимо проверить структуру папок и файлы.")
    return pd.DataFrame()

  if missing_files:
    print(f"Не удалось загрузить файлов: {len(missing_files)}")

  df = pd.concat(all_chunks, ignore_index=True)
  print(f"Успешно загруженные файлы: {len(all_chunks)}")
  return df

df = load_timesheet_data(BASE_DIR, Config.WEEKS_RANGE, Config.FILE_PATTERN)

## 3. Очистка и нормализация схемы данных

1. Переименовываем столбцы в snake_case
2. Удаляем технические столбцы

In [ ]:
COLUMN_MAPPING = {
    'Unnamed: 0': 'week_label',
    'Unnamed: 1': 'date',
    'С': 'start_time',
    'По': 'end_time',
    'Часы': 'duration_hours',
    'Описание': 'description',
    'Тип Активности': 'activity_type'
}

In [ ]:
def preprocess_df(df, mapping):
  if df.empty:
    return df

  # Ренэйминг соглсно мееппингу тех объектов, которые есть в df
  existing_mapping = {k: v for k, v in mapping.items() if k in df.columns}
  df = df.rename(columns=existing_mapping)

  # Оставляем только нужные колонки
  target_columns = list(existing_mapping.values())
  if 'source_file' in df.columns:
    target_columns.append('source_file')

  df = df[target_columns].copy()

  return df

df = preprocess_df(df, COLUMN_MAPPING)

## 4. Базовый препроцессинг и приведение типов

### 4.1 Преобразование типов
* Преобразование `date`, `start_time` и `end_time` в формат `datetime`
* Преобразование типа `object` в `stringDtype`

In [ ]:
def cast_types_schema(df):
  if df.empty:
    return df

  df = df.copy()

  df["date"] = pd.to_datetime(df["date"], format="%d %b %y", errors="coerce")

  for col in ["start_time", "end_time"]:
    df[col] = pd.to_datetime(df[col], format="%H:%M", errors="coerce").dt.time

  for col in ["start_time", "end_time"]:
    df[col] = df.apply(
        lambda x: pd.Timestamp.combine(x["date"].date(), x[col])
        if pd.notnull(x["date"]) and pd.notnull(x[col])
        else pd.NaT,
        axis=1
    )

  for col in ["week_label", "description", "activity_type"]:
    df[col] = df[col].astype("string")

  return df

df = cast_types_schema(df)


### 4.2 Нормализация текстовых описаний

In [ ]:
def normalize_text_fields(df):
  if df.empty:
    return df

  df = df.copy()

  for col in ["description", "activity_type"]:
    df[col] = df[col].str.strip().str.lower()

  return df

df = normalize_text_fields(df)


## 5. Feature Engineering

Создадим аналитические метрики паттернов эффективности, оценки когнективной нагрузки

In [ ]:
def build_feature_pipeline(df):
  if df.empty:
    return df

  df = df.copy()

  # === Временные признаки ===
  df["hour_start"] = df["start_time"].dt.hour
  df["day_name"] = df["date"].dt.day_name()
  df["is_weekend"] = df["date"].dt.dayofweek.isin([5, 6]).astype(int)
  df["is_overtime"] = (df["end_time"].dt.hour >= 19).astype(int)

  return df

df = build_feature_pipeline(df)
